In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

In [2]:
import lsst.sims.maf.db as db
import lsst.sims.maf.metricBundles as mmb
import lsst.sims.maf.plots as plots
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.batches as batches
from lsst.sims.maf.runComparison import RunComparison

/ssd/lsst/lsst_repos/sims_maf/python/lsst/sims/maf/runComparison/runComparison.py:23: UserWarning: 
The generateDiffHtml method requires bokeh to be installed
but it is not needed to use the other methods in this class.
Run: pip install bokeh then restart your jupyter notebook kernel.
  'but it is not needed to use the other methods in this class.'+'\n'+


In [3]:
def read_all_metricdata(workDir, metadata):
    # Just read in ALL metrics in the (joint or single) directory, 
    # Note that this will be REALLY SLOW if you give it many directories.
    if metadata is None:
        matchstring = os.path.join(workDir, '*.npz')
    else:
        matchstring = os.path.join(workDir, f'*{metadata}*.npz')
    metricfiles = glob.glob(matchstring)
    metricNames = []
    for m in metricfiles:
        mname = os.path.split(m)[-1].replace('.npz', '')
        metricNames.append(mname)

    bdict = {}
    for mName, mFile in zip(metricNames, metricfiles):
        bdict[mName] = mmb.createEmptyMetricBundle()
        bdict[mName].read(mFile)

    first = bdict[metricNames[0]]
    figroot = f'{first.runName}'
    if metadata is not '.':
        figroot += f'_{metadata}'
    return figroot, bdict

def read_metricdata(workDir, metricName, verbose=True):
    # Just read a specific metric output which matches the pattern of metricName
    matchstring = os.path.join(workDir, f'*{metricName}*.npz')
    metricfiles = glob.glob(matchstring)
    if len(metricfiles) > 1:
        if verbose:
            print(f'Something is maybe wrong - found {len(metricfiles)} files matching {metricName} in {workDir}')
        return
    if len(metricfiles) == 0:
        if verbose:
            print(f'Something is maybe wrong - found no metricfiles matching {metricName} in {workDir}')
        return
    bundle = mmb.createEmptyMetricBundle()
    bundle.read(metricfiles[0])
    return bundle

In [4]:
rootDirs = {}
rootDirs['1.4'] = 'fbs_1.4'
rootDirs['1.5'] = 'fbs_1.5'
rootDirs['1.6'] = 'fbs_1.6'
workDirs = {}
runNames = {}
for root in rootDirs:
    workDirs[root] = {}
    runNames[root] = [os.path.split(i.replace('/science', ''))[-1] for i in 
                       glob.glob(os.path.join(rootDirs[root], '*_10yrs', 'science'))]
    for run in runNames[root]:
        workDirs[root][run] = os.path.join(rootDirs[root], run, 'science')
print(len(runNames['1.4']), len(runNames['1.5']), len(runNames['1.6']))
print(len(workDirs['1.4']), len(workDirs['1.5']), len(workDirs['1.6']))

118 86 30
118 86 30


In [5]:
outdir = 'science_comp'
!/bin/rm -r $outdir
!mkdir $outdir
ph = plots.PlotHandler(outDir=outdir, figformat='png', dpi=630, savefig=False)

In [6]:
rc = {}
for root in rootDirs:
    rc[root] = RunComparison(baseDir='.', runNames=runNames[root], rundirs=list(workDirs[root].values()))

In [7]:
mdict = rc['1.4'].buildMetricDict()

In [8]:
len(list(mdict.keys()))

151

In [9]:
for root in rc:
    rc[root].addSummaryStats(mdict)

In [10]:
for root in rc:
    rc[root].summaryStats.to_csv(os.path.join(outdir, f'sci_summary_{root.replace(".", "_")}.csv'))

In [11]:
!ls $outdir

sci_summary_1_4.csv  sci_summary_1_5.csv  sci_summary_1_6.csv


In [12]:
outdir = 'meta_comp'
!/bin/rm -r $outdir
!mkdir $outdir
ph = plots.PlotHandler(outDir=outdir, figformat='png', dpi=630, savefig=False)

In [13]:
rootDirs = {}
rootDirs['1.4'] = 'fbs_1.4'
rootDirs['1.5'] = 'fbs_1.5'
rootDirs['1.6'] = 'fbs_1.6'
workDirs = {}
runNames = {}
for root in rootDirs:
    workDirs[root] = {}
    runNames[root] = [os.path.split(i.replace('/metadata', ''))[-1] for i in 
                       glob.glob(os.path.join(rootDirs[root], '*_10yrs', 'metadata'))]
    for run in runNames[root]:
        workDirs[root][run] = os.path.join(rootDirs[root], run, 'metadata')
print(len(runNames['1.4']), len(runNames['1.5']), len(runNames['1.6']))
print(len(workDirs['1.4']), len(workDirs['1.5']), len(workDirs['1.6']))

rc = {}
for root in rootDirs:
    rc[root] = RunComparison(baseDir='.', runNames=runNames[root], rundirs=list(workDirs[root].values()))

118 85 30
118 85 30


In [14]:
for root in rootDirs:
    mdict = rc[root].buildMetricDict()
    rc[root].addSummaryStats(mdict)
    rc[root].summaryStats.to_csv(os.path.join(outdir, f'meta_summary_{root.replace(".", "_")}.csv'))

/ssd/lsst/lsst_repos/sims_maf/python/lsst/sims/maf/runComparison/runComparison.py:278: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  stats = pd.concat(tempDFList)


In [16]:
rc['1.5'].summaryStats

,fOArea fO All visits HealpixSlicer,fOArea/benchmark fO All visits HealpixSlicer,fONv MedianNvis fO All visits HealpixSlicer,fONv MinNvis fO All visits HealpixSlicer,fONv/benchmark MedianNvis fO All visits HealpixSlicer,fONv/benchmark MinNvis fO All visits HealpixSlicer,Median Parallax Error (18k) Parallax Error @ 22.4 All visits HealpixSlicer,95th Percentile Parallax Error Parallax Error @ 22.4 All visits HealpixSlicer,Mean Parallax Error @ 22.4 All visits HealpixSlicer,Rms Parallax Error @ 22.4 All visits HealpixSlicer,...,Max Nvisits DD:EDFSb OneDSlicer,Mean Nvisits DD:EDFSb OneDSlicer,Median Nvisits DD:EDFSb OneDSlicer,Min Nvisits DD:EDFSb OneDSlicer,N(+3Sigma) Nvisits DD:EDFSb OneDSlicer,N(-3Sigma) Nvisits DD:EDFSb OneDSlicer,Nvisits DD:EDFSb OneDSlicer,Rms Nvisits DD:EDFSb OneDSlicer,Nvisits DD:EDFSb,Fraction of total Nvisits DD:EDFSb
baseline_samefilt_v1.5_10yrs,18304.155108,1.016898,1003.0,927.0,1.215758,1.123636,2.074618,8.962334,4.197896,15.031543,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
agnddf_v1.5_10yrs,18200.082696,1.011116,960.0,890.0,1.163636,1.078788,2.050710,10.323067,4.525217,17.706578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
alt_dust_v1.5_10yrs,17744.346247,0.985797,898.0,791.0,1.088485,0.958788,2.234965,12.765718,4.860301,8.661743,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
baseline_v1.5_10yrs,18217.707863,1.012095,967.0,897.0,1.172121,1.087273,2.064350,10.390527,4.459374,12.939890,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bulges_bs_v1.5_10yrs,18866.481851,1.048138,872.0,834.0,1.056970,1.010909,2.226068,12.543737,4.880610,7.946569,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wfd_depth_scale0.75_noddf_v1.5_10yrs,18324.298156,1.018017,899.0,840.0,1.089697,1.018182,2.109746,5.312819,3.279112,10.407259,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
wfd_depth_scale0.80_v1.5_10yrs,18049.849134,1.002769,894.0,827.0,1.083636,1.002424,2.143197,6.275822,3.547748,9.849148,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
goodseeing_i_v1.5_10yrs,18214.350688,1.011908,966.0,893.0,1.170909,1.082424,2.060524,10.431037,4.528746,18.193229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
roll_mod2_dust_sdf_0.20_v1.5_10yrs,17609.219970,0.978290,891.0,780.0,1.080000,0.945455,2.338367,11.525191,4.623486,6.632347,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
rolling_mod2_sdf_0.20_v1.5_10yrs,18199.243403,1.011069,964.0,890.0,1.168485,1.078788,2.147173,9.604034,4.357261,14.889079,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
